In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c word2vec-21
!unzip word2vec-21.zip

Archive:  word2vec-21.zip
  inflating: sample_answers.csv      
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()  # <<< 添加这一行
# Load data
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # 使用 TreebankWordTokenizer 分词
    words = tokenizer.tokenize(text.lower())
    # Keep only alphabetic tokens
    words = [word for word in words if word.isalpha()]
    # Remove English stopwords
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if w not in stop_words]
    return words

# Generate list of tokenized sentences for Word2Vec training
sentences = [clean_text(review) for review in train_df["review"]]

# Train Word2Vec model
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,     # Dimensionality of the word embeddings
    window=5,            # Context window size
    min_count=1,         # Minimum word frequency
    workers=4,           # Number of CPU cores
    sg=1               # Use Skip-Gram (1) or CBOW (0)
)

# Function to generate average sentence embedding
def get_sentence_embedding(text, model):
    words = clean_text(text)
    vectors = [model.wv[word] for word in words if word in model.wv]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Generate embeddings for train and test datasets
X_train = np.array([get_sentence_embedding(review, w2v_model) for review in train_df["review"]])
y_train = train_df["label"].values
X_test = np.array([get_sentence_embedding(review, w2v_model) for review in test_df["review"]])

# Train a Logistic Regression classifier using a pipeline with standardization
clf = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=1000, random_state=42)
)
clf.fit(X_train, y_train)

# Predict on test set
test_predictions = clf.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    "Id": range(len(test_df)),  # 自动生成 ID
    "Predicted": test_predictions
})
submission.to_csv("submission.csv", index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
